# MPFIT_first-fit_only

In [ ]:
import os
import sys
import numpy as np
np.set_printoptions(threshold=sys.maxsize)    # This will force print numpy arrays without truncation

os.chdir('../')
from config import CONFIG_INI
import hstmarg
from limb_darkening import limb_dark_fit
#from presto_mpfit import mpfit
from mgefit.cap_mpfit import mpfit

In [ ]:
# Load data paths
localDir = CONFIG_INI.get('data_paths', 'local_path')
outDir = os.path.join(localDir, CONFIG_INI.get('data_paths', 'output_path'))
curr_model = CONFIG_INI.get('data_paths', 'current_model')
dataDir = os.path.join(localDir, os.path.join(localDir, CONFIG_INI.get('data_paths', 'data_path')), curr_model)

# Read in the txt files for the lightcurve data
x, y, err, sh = np.loadtxt(os.path.join(dataDir, 'W17_white_lightcurve_test_data.txt'), skiprows=7, unpack=True)
wavelength = np.loadtxt(os.path.join(dataDir, 'W17_wavelength_test_data.txt'), skiprows=3)

print("dataDir:", dataDir)
print("outdir:", outDir)
print("x.shape:", x.shape)
print("y.shape:", y.shape)
print("err.shape:", err.shape)
print("sh.shape:", sh.shape)

In [ ]:
    # READ THE CONSTANTS
    Gr = CONFIG_INI.getfloat('constants', 'big_G')
    day_to_sec = CONFIG_INI.getfloat('constants', 'dtosec')
    HST_period = CONFIG_INI.getfloat('constants', 'HST_period')

    # We want to keep the raw data as is, so we generate helper arrays that will get changed from model to model
    img_date = x    # time array
    img_flux = y    # flux array
    flux0 = img_flux[0]   # first flux data point
    T0 = img_date[0]      # first time data point
    nexposure = len(img_date)   # Total number of exposures in the observation

    # READ IN THE PLANET STARTING PARAMETERS
    rl = CONFIG_INI.getfloat('planet_parameters', 'rl')          # Rp/R* estimate
    epoch = CONFIG_INI.getfloat('planet_parameters', 'epoch')    # center of transit time in MJD
    inclin = CONFIG_INI.getfloat('planet_parameters', 'inclin') * ((2 * np.pi) / 360)   # inclination, converting it to radians
    ecc = CONFIG_INI.getfloat('planet_parameters', 'ecc')                            # eccentricity
    omega = CONFIG_INI.getfloat('planet_parameters', 'omega') * ((2 * np.pi) / 360)    # orbital omega, converting it to radians
    Per = CONFIG_INI.getfloat('planet_parameters', 'Per') * day_to_sec               # period in seconds

    constant1 = ((Gr * np.square(Per)) / (4 * np.square(np.pi))) ** (1 / 3)
    aor = CONFIG_INI.getfloat('planet_parameters', 'aor')
    MsMpR = (aor / constant1) ** 3.                          # density of the system

    # SET THE STARTING PARAMETERS FOR THE SYSTEMATIC MODELS
    m_fac = 0.0  # Linear Slope
    HSTP1 = 0.0  # Correct HST orbital phase
    HSTP2 = 0.0  # Correct HST orbital phase^2
    HSTP3 = 0.0  # Correct HST orbital phase^3
    HSTP4 = 0.0  # Correct HST orbital phase^4
    xshift1 = 0.0  # X-shift in wavelength
    xshift2 = 0.0  # X-shift in wavelength^2
    xshift3 = 0.0  # X-shift in wavelength^3
    xshift4 = 0.0  # X-shift in wavelength^4

In [ ]:
# LIMB DARKENING
# NEW: Implement a suggestion for the user to use 3D if his parameters match the options available in the 3D models

M_H = CONFIG_INI.getfloat('limb_darkening', 'metallicity')    # metallicity
Teff = CONFIG_INI.getfloat('limb_darkening', 'Teff')   # effective temperature
logg = CONFIG_INI.getfloat('limb_darkening', 'logg')   # log(g), gravitation

# Define limb darkening directory, which is inside this package
limbDir = os.path.join('..', 'Limb-darkening')
ld_model = CONFIG_INI.get('limb_darkening', 'ld_model')
grat = CONFIG_INI.get('technical_parameters', 'grating')
uLD, c1, c2, c3, c4, cp1, cp2, cp3, cp4, aLD, bLD = limb_dark_fit(grat, wavelength, M_H, Teff, logg, limbDir,
                                                                  ld_model)

In [ ]:
# PLACE ALL THE PRIORS IN AN ARRAY
# p0 =        [0,    1,     2,      3,     4,    5,    6,    7,  8,  9,  10, 11, 12,  13,    14,    15,    16,    17,     18,      19,      20,      21   ]
p0 = np.array([rl, flux0, epoch, inclin, MsMpR, ecc, omega, Per, T0, c1, c2, c3, c4, m_fac, HSTP1, HSTP2, HSTP3, HSTP4, xshift1, xshift2, xshift3, xshift4])

# Create an array with the names of the priors
p0_names = np.array(['rl', 'flux0', 'epoch', 'inclin', 'MsMpR', 'ecc', 'omega', 'Per', 'T0', 'c1', 'c2', 'c3', 'c4',
                     'm_fac', 'HSTP1', 'HSTP2', 'HSTP3', 'HSTP4', 'xshift1', 'xshift2', 'xshift3', 'xshift4'])

# Create a dictionary for easier use in calculations
p0_dict = {key: val for key, val in zip(p0_names, p0)}

In [ ]:
# SELECT THE SYSTEMATIC GRID OF MODELS TO USE
# 1 in the grid means the parameter is fixed, 0 means it is free
grid_selection = CONFIG_INI.get('technical_parameters', 'grid_selection')
grid = hstmarg.wfc3_systematic_model_grid_selection(grid_selection)
nsys, nparams = grid.shape   # nsys = number of systematic models, nparams = number of parameters
print("grid.shape:", grid.shape)

## Single first MPFIT run

In [ ]:
# what systematic model to use:
s = 37

In [ ]:
print('SYSTEMATIC MODEL {} of {}'.format(s+1, nsys))
systematics = grid[s, :]
print(systematics)

#print_dict = {name: fix for name, fix in zip(p0_names, systematics)}   # this is just for printing purposes
#print(print_dict)

In [ ]:
# Create two dictionaries in which each parameter in p0 gets some extra parameters assigned, which we then feed
# into mpfit. This dictionary has the sole purpose of preparing the input data for mpfit in such a way that
# it works.
parinfo = []
for i, value in enumerate(p0):
    info = {'value': 0., 'fixed': 0, 'limited': [0, 0], 'limits': [0., 0.]}
    info['value'] = value
    info['fixed'] = systematics[i]
    parinfo.append(info)
    print(info)
fa = {'x': img_date, 'y': img_flux, 'err': err, 'sh': sh}

print('\n')
print(fa.keys())
print('\n')


In [ ]:
# Run one round of MPFIT
print('\nSTART MPFIT\n')
mpfit_result = mpfit(hstmarg.transit_circle, functkw=fa, parinfo=parinfo, quiet=True)

print('\nTHIS ROUND OF MPFIT IS DONE\n')

In [ ]:
# Get the error array
pcerror = mpfit_result.perror 
print("systematics:", systematics)
print(pcerror)